In [1]:
import tensorflow as tf
# import plotly as px
import pandas as pd
# import matplotlib.pyplot as plt
import numpy as np
# import plotly.express as px
import copy
# from Modules.Technical_indicators import ATR, BB, RSI, ADX, MACD
# from plotly.subplots import make_subplots
# import plotly.graph_objs as go
# from Modules.Plotting import Compare_Plot, Compare_Plot_bar
import warnings
warnings.filterwarnings("ignore")



2024-04-22 07:39:54.018096: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 07:39:54.018230: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 07:39:54.134561: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
df = pd.read_csv(r'/kaggle/input/btc-usd/BTC-USD.csv')
df['Date'] = pd.to_datetime(df['Date'])
df_time = df.set_index('Date')

In [3]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date',inplace = True)
data = copy.deepcopy(df)
df.dropna(inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3503 entries, 2014-09-17 to 2024-04-19
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       3503 non-null   float64
 1   High       3503 non-null   float64
 2   Low        3503 non-null   float64
 3   Close      3503 non-null   float64
 4   Adj Close  3503 non-null   float64
 5   Volume     3503 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 191.6 KB


In [4]:
df = df.drop(columns = ['Open','High','Low','Close','Volume'],axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3503 entries, 2014-09-17 to 2024-04-19
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Adj Close  3503 non-null   float64
dtypes: float64(1)
memory usage: 54.7 KB


In [5]:
labels_list = []
feature_list = []
window_size = 30
horizon = 1
feature_array = np.array(df)
for i in range(0, len(df['Adj Close'])-(window_size+horizon-1)):
    feature_list.append(np.array(feature_array[i:i+window_size]))
    labels_list.append(np.array(df['Adj Close'][i+window_size:i+window_size+horizon]))

# Convert the list of arrays to a numpy array
labels = np.array(labels_list)
feature = np.array(feature_list)

labels  = tf.convert_to_tensor(labels)
feature = tf.convert_to_tensor(feature)

In [6]:
labels.shape, feature.shape

(TensorShape([3473, 1]), TensorShape([3473, 30, 1]))

In [9]:
def make_train_test_splits(windows, labels, test_split = 0.15):
    split_size = int(len(windows)* (1-test_split))
    train_windows = windows[:split_size]
    train_labels = labels[:split_size]
    test_windows = windows[split_size:]
    test_labels = labels[split_size:]
    return train_windows, test_windows, train_labels, test_labels

In [10]:
train_windows, test_windows, train_labels, test_labels = make_train_test_splits(feature,labels)
train_windows.shape, (test_windows).shape, (train_labels).shape, (test_labels).shape

(TensorShape([2952, 30, 1]),
 TensorShape([521, 30, 1]),
 TensorShape([2952, 1]),
 TensorShape([521, 1]))

In [11]:
batch_size = 32
def prepare_data_for_training(X_train, X_test,Y_train, Y_test, batch_size = 32):

    train_text_dataset = tf.data.Dataset.from_tensor_slices(X_train)
    train_labels_dataset = tf.data.Dataset.from_tensor_slices(Y_train)

    train_dataset = tf.data.Dataset.zip((train_text_dataset, train_labels_dataset))
    train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    test_text_dataset = tf.data.Dataset.from_tensor_slices(X_test)
    test_labels_dataset = tf.data.Dataset.from_tensor_slices(Y_test)

    test_dataset = tf.data.Dataset.zip((test_text_dataset, test_labels_dataset))
    test_dataset = test_dataset.shuffle(buffer_size=len(X_test)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return train_dataset, test_dataset

In [12]:
train_dataset, val_dataset = prepare_data_for_training(X_train = train_windows, X_test= test_windows,Y_train = train_labels, Y_test =  test_labels , batch_size = 32)

In [ ]:
# batch_size = 32
# train_text_dataset = tf.data.Dataset.from_tensor_slices(feature)
# train_labels_dataset = tf.data.Dataset.from_tensor_slices(labels)
# train_dataset = tf.data.Dataset.zip((train_text_dataset, train_labels_dataset))
# train_dataset = train_dataset.shuffle(buffer_size=len(feature)).batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
# callback = EarlyStopping(
#     monitor="loss",#quantity to monitor
#     min_delta=0.0001,#Minimum change in the monnitored quantity as an improvement
#     patience=20,#No of epochs with no improvement
#     verbose=1,
#     mode="auto",
#     restore_best_weights=True
# )

In [20]:
def create_callbacks(dir_name, experiment_name):
    """
    Creates callbacks for training.

    Args:
    - dir_name: target directory to store TensorBoard log files and model checkpoints
    - experiment_name: name of experiment directory

    Returns:
    - callbacks: list of callbacks
    """
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    print(f"Saving TensorBoard log files to: {log_dir}")

    # TensorBoard callback for logging
    tensorboard_callback = TensorBoard(log_dir=log_dir)

    # ModelCheckpoint callback for saving the best model based on validation loss
    model_checkpoint_callback = ModelCheckpoint(filepath=f"{dir_name}/{experiment_name}/best_model.keras",
                                                monitor='val_loss',  # Monitor validation loss
                                                save_best_only=True,
                                                verbose=1)

    return [tensorboard_callback, model_checkpoint_callback]



### N_BEATS custom layers

In [ ]:
# class NBeatsBlock(tf.keras.layers.Layer):
#     def __init__(self,
#                 input_size:int,
#                 theta_size:int,
#                 horizon:int,
#                 n_neurons:int,
#                 n_layers:int,
#                 **kwargs):

#         super().__init__(**kwargs)

#         # Initialize attributes based on input parameters
#         self.input_size = input_size  # Size of the input time series
#         self.theta_size = theta_size  # Size of the theta vector
#         self.horizon = horizon  # Forecasting horizon
#         self.n_neurons = n_neurons  # Number of neurons in each hidden layer
#         self.n_layers = n_layers  # Number of hidden layers

#         # Define hidden layers with ReLU activation
#         self.hidden = [tf.keras.layers.Dense(n_neurons, activation="relu") for _ in range(n_layers)]

#         # Define theta layer with linear activation
#         self.theta_layer = tf.keras.layers.Dense(theta_size, activation="linear", name="theta")

#     def call(self, inputs):
#         x = inputs  # Input tensor
#         # Pass input through each hidden layer sequentially
#         for layer in self.hidden:
#             x = layer(x)
        
#         # Compute theta vector by passing output of last hidden layer through theta layer
#         theta = self.theta_layer(x)

#         # Split theta vector into backcast and forecast components
#         backcast, forecast = theta[:, :self.input_size], theta[:, -self.horizon:]

#         # Return backcast and forecast predictions
#         return backcast, forecast


In [ ]:
# N_EPOCHS = 1000
# N_NEURONS = 512
# N_LAYERS  = 4
# N_STACKS = 30

# input_size = window_size * horizon
# theta_size = window_size + horizon

# input_size, theta_size

In [ ]:
# window_size = 30
# horizon = 1

# class NBeatsBlock(tf.keras.layers.Layer):
#     def __init__(self,
#                 input_size:int,
#                 theta_size:int,
#                 horizon:int,
#                 n_neurons:int,
#                 n_layers:int,
#                 **kwargs):

#         super().__init__(**kwargs)

#         # Initialize attributes based on input parameters
#         self.input_size = input_size  # Size of the input time series
#         self.theta_size = theta_size  # Size of the theta vector
#         self.horizon = horizon  # Forecasting horizon
#         self.n_neurons = n_neurons  # Number of neurons in each hidden layer
#         self.n_layers = n_layers  # Number of hidden layers

#         # Define hidden layers with ReLU activation
#         self.hidden = [tf.keras.layers.Dense(n_neurons, activation="relu") for _ in range(n_layers)]

#         # Define theta layer with linear activation
#         self.theta_layer = tf.keras.layers.Dense(theta_size, activation="linear", name="theta")

#     def call(self, inputs):
#         x = inputs  # Input tensor
#         # Pass input through each hidden layer sequentially
#         for layer in self.hidden:
#             x = layer(x)
        
#         # Compute theta vector by passing output of last hidden layer through theta layer
#         theta = self.theta_layer(x)

#         # Split theta vector into backcast and forecast components
#         backcast, forecast = theta[:, :self.input_size], theta[:, -self.horizon:]

#         # Return backcast and forecast predictions
#         return backcast, forecast



# N_EPOCHS = 1000
# N_NEURONS = 512
# N_LAYERS  = 4
# N_STACKS = 30

# input_size = window_size * horizon
# theta_size = window_size + horizon

# from tensorflow.keras import layers 
# tf.random.set_seed(42)

# # Initialize the first N-Beats block layer
# nbeat_block_layer = NBeatsBlock(
#                                 input_size=input_size,
#                                 theta_size=theta_size,
#                                 horizon=horizon,
#                                 n_neurons=N_NEURONS,
#                                 n_layers=N_LAYERS,
#                                 name='Initial_block'
#                                 )

# # Define input layer for the stacked model
# stack_input = layers.Input(shape=(input_size), name='stack_input')

# # Apply the initial N-Beats block layer to the input layer
# backcast, forecast = nbeat_block_layer(stack_input)

# # Calculate residuals by subtracting the backcast predictions from the input
# residuals = layers.subtract([stack_input, backcast], name="subtract_00")

# # Iterate over the remaining N-Beats blocks to create a stacked architecture
# for i, _ in enumerate(range(N_STACKS - 1)):
#     # Apply N-Beats block to residuals from the previous block
#     backcast, block_forecast = NBeatsBlock(
#                                 input_size=input_size,
#                                 theta_size=theta_size,
#                                 horizon=horizon,
#                                 n_neurons=N_NEURONS,
#                                 n_layers=N_LAYERS,
#                                 name=f'NBeatsBlock_{i}'
#                                 )(residuals)

#     # Update residuals by subtracting backcast predictions
#     residuals = layers.subtract([residuals, backcast], name=f"subtract_{i}")
#     # Update forecast by adding block's forecast predictions
#     forecast = layers.add([forecast, block_forecast], name=f"add_{i}")

# # Define the N-BEATS model using the input and output layers
# N_BEATS_MODEL = tf.keras.Model(
#                                 inputs=stack_input,
#                                 outputs=forecast,
#                                 name='Model_N_BEATS'
#                                 )

# # Compile the model with Mean Absolute Error loss, Adam optimizer, and additional metrics
# N_BEATS_MODEL.compile(loss='mae', optimizer=tf.keras.optimizers.Adam(0.001), metrics=["mae", "mse"])

# # Display the summary of the model architecture
# N_BEATS_MODEL.summary()


In [18]:
window_size = 30
horizon = 1

class NBeatsBlock(tf.keras.layers.Layer):
    def __init__(self,
                input_size:int,
                theta_size:int,
                horizon:int,
                n_neurons:int,
                n_layers:int,
                **kwargs):

        super().__init__(**kwargs)

        # Initialize attributes based on input parameters
        self.input_size = input_size  # Size of the input time series
        self.theta_size = theta_size  # Size of the theta vector
        self.horizon = horizon  # Forecasting horizon
        self.n_neurons = n_neurons  # Number of neurons in each hidden layer
        self.n_layers = n_layers  # Number of hidden layers

        # Define hidden layers with ReLU activation
        self.hidden = [tf.keras.layers.Dense(n_neurons, activation="relu") for _ in range(n_layers)]

        # Define theta layer with linear activation
        self.theta_layer = tf.keras.layers.Dense(theta_size, activation="linear", name="theta")

    def call(self, inputs):
        x = inputs  # Input tensor
        # Pass input through each hidden layer sequentially
        for layer in self.hidden:
            x = layer(x)
        
        # Compute theta vector by passing output of last hidden layer through theta layer
        theta = self.theta_layer(x)

        # Split theta vector into backcast and forecast components
        backcast, forecast = theta[:, :self.input_size], theta[:, -self.horizon:]

        # Return backcast and forecast predictions
        return backcast, forecast



N_EPOCHS = 1000
N_NEURONS = 512
N_LAYERS  = 4
N_STACKS = 30

input_size = window_size * horizon
theta_size = window_size + horizon

from tensorflow.keras import layers 
tf.random.set_seed(42)

# Initialize the first N-Beats block layer
nbeat_block_layer = NBeatsBlock(
                                input_size=input_size,
                                theta_size=theta_size,
                                horizon=horizon,
                                n_neurons=N_NEURONS,
                                n_layers=N_LAYERS,
                                name='Initial_block'
                                )

# Define input layer for the stacked model
stack_input = layers.Input(shape=(input_size,), name='stack_input')

# Apply the initial N-Beats block layer to the input layer
backcast, forecast = nbeat_block_layer(stack_input)

# Calculate residuals by subtracting the backcast predictions from the input
residuals = layers.subtract([stack_input, backcast], name="subtract_00")

# Iterate over the remaining N-Beats blocks to create a stacked architecture
for i, _ in enumerate(range(N_STACKS - 1)):
    # Apply N-Beats block to residuals from the previous block
    backcast, block_forecast = NBeatsBlock(
                                input_size=input_size,
                                theta_size=theta_size,
                                horizon=horizon,
                                n_neurons=N_NEURONS,
                                n_layers=N_LAYERS,
                                name=f'NBeatsBlock_{i}'
                                )(residuals)

    # Update residuals by subtracting backcast predictions
    residuals = layers.subtract([residuals, backcast], name=f"subtract_{i}")
    # Update forecast by adding block's forecast predictions
    forecast = layers.add([forecast, block_forecast], name=f"add_{i}")

# Define the N-BEATS model using the input and output layers
N_BEATS_MODEL = tf.keras.Model(
                                inputs=stack_input,
                                outputs=forecast,
                                name='Model_N_BEATS'
                                )

# Compile the model with Mean Absolute Error loss, Adam optimizer, and additional metrics
N_BEATS_MODEL.compile(loss='mae', optimizer=tf.keras.optimizers.Adam(0.001), metrics=["mae", "mse"])

# Display the summary of the model architecture
N_BEATS_MODEL.summary()


Model: "Model_N_BEATS"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ stack_input         │ (None, 30)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Initial_block       │ [(None, 30),      │    819,743 │ stack_input[0][0] │
│ (NBeatsBlock)       │ (None, 1)]        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ subtract_00         │ (None, 30)        │          0 │ stack_input[0][0… │
│ (Subtract)          │                   │            │ Initial_block[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ NBeatsBlock_0       │ [(None, 30),      │    819,743 │ subtract_00[0][0] │
│ (NBeatsBlock)       │ (None, 1)]        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ subtract_0          │ (None, 30)        │          0 │ subtract_00[0][0… │
│ (Subtract)          │                   │            │ NBeatsBlock_0[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ NBeatsBlock_1       │ [(None, 30),      │    819,743 │ subtract_0[0][0]  │
│ (NBeatsBlock)       │ (None, 1)]        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ subtract_1          │ (None, 30)        │          0 │ subtract_0[0][0], │
│ (Subtract)          │                   │            │ NBeatsBlock_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ NBeatsBlock_2       │ [(None, 30),      │    819,743 │ subtract_1[0][0]  │
│ (NBeatsBlock)       │ (None, 1)]        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ subtract_2          │ (None, 30)        │          0 │ subtract_1[0][0], │
│ (Subtract)          │                   │            │ NBeatsBlock_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ NBeatsBlock_3       │ [(None, 30),      │    819,743 │ subtract_2[0][0]  │
│ (NBeatsBlock)       │ (None, 1)]        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ subtract_3          │ (None, 30)        │          0 │ subtract_2[0][0], │
│ (Subtract)          │                   │            │ NBeatsBlock_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ NBeatsBlock_4       │ [(None, 30),      │    819,743 │ subtract_3[0][0]  │
│ (NBeatsBlock)       │ (None, 1)]        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ subtract_4          │ (None, 30)        │          0 │ subtract_3[0][0], │
│ (Subtract)          │                   │            │ NBeatsBlock_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ NBeatsBlock_5       │ [(None, 30),      │    819,743 │ subtract_4[0][0]  │
│ (NBeatsBlock)       │ (None, 1)]        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ subtract_5          │ (None, 30)        │          0 │ subtract_4[0][0], │
│ (Subtract)          │                   │            │ NBeatsBlock_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ NBeatsBlock_6       │ [(None, 30),      │    819,743 │ subtract_5[0][0]  │
│ (NBeatsBlock)       │ (None, 1)]        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ subtract_6          │ (None, 30)        │          0 │ subtract_5[0][0]

 Total params: 24,592,290 (93.81 MB)

 Trainable params: 24,592,290 (93.81 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
N_BEATS_MODEL.fit(train_dataset, 
                  epochs=N_EPOCHS,validation_data = val_dataset,
                  callbacks=[
                      tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=100, restore_best_weights=True),
                      tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=50, verbose=1),
                      create_callbacks('Model', N_BEATS_MODEL.name)
                  ])


Saving TensorBoard log files to: Model/Model_N_BEATS/20240422-074513
Epoch 1/1000


I0000 00:00:1713771966.558473      82 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - loss: 28260.6348 - mae: 28260.6348 - mse: 19966224384.0000
Epoch 1: val_loss improved from inf to 4142.19971, saving model to Model/Model_N_BEATS/best_model.keras
93/93 ━━━━━━━━━━━━━━━━━━━━ 79s 281ms/step - loss: 28052.1465 - mae: 28052.1465 - mse: 19806924800.0000 - val_loss: 4142.1997 - val_mae: 4142.1997 - val_mse: 20811308.0000 - learning_rate: 0.0010
Epoch 2/1000
92/93 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 1153.8470 - mae: 1153.8470 - mse: 5195560.5000
Epoch 2: val_loss improved from 4142.19971 to 2249.90894, saving model to Model/Model_N_BEATS/best_model.keras
93/93 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - loss: 1148.8187 - mae: 1148.8187 - mse: 5158460.0000 - val_loss: 2249.9089 - val_mae: 2249.9089 - val_mse: 7125272.5000 - learning_rate: 0.0010
Epoch 3/1000
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 793.9354 - mae: 793.9354 - mse: 2716549.2500
Epoch 3: val_loss improved from 2249.90894 to 2116.54199, saving model to Model/Model_N_

In [ ]:
def mean_absolute_scaled_error(y_true, y_pred, seasonality=1):
    mae = np.mean(np.abs(y_true - y_pred))
    naive_forecast = y_true[:-seasonality]
    naive_mae = np.mean(np.abs(naive_forecast - y_true[seasonality:]))
    mase = mae / naive_mae
    return mase

def evaluate_preds(y_true, y_pred):
  # Make sure float32 (for metric calculations)
  y_true = tf.cast(y_true, dtype=tf.float32)
  y_pred = tf.cast(y_pred, dtype=tf.float32)

  # Calculate various metrics
  mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
  mse = tf.keras.metrics.mean_squared_error(y_true, y_pred)
  rmse = tf.sqrt(mse)
  huber_loss = tf.keras.losses.Huber()(y_true, y_pred)
  mase = mean_absolute_scaled_error(y_true.numpy(), y_pred.numpy(),seasonality=1)

  # Account for different sized metrics (for longer horizons, reduce to single number)
  if mae.ndim > 0: # if mae isn't already a scalar, reduce it to one by aggregating tensors to mean
    mae = tf.reduce_mean(mae)
    mse = tf.reduce_mean(mse)
    rmse = tf.reduce_mean(rmse)
    huber_loss = tf.reduce_mean(huber_loss)
    mase = tf.reduce_mean(mase)

  print("For mae, mse, rmse and hubber : lower value is better.\nmase :- A scaled eror is >1 if the forecast is worse than the naive and <1 if the forecast is better that the naive.")
  print(f"""
    "mae": {mae.numpy()},
    "mse": {mse.numpy()},
    "rmse": {rmse.numpy()},
    "huber_loss": {huber_loss.numpy()},
    "mase": {mase}
    """)
  return {
        "mae": mae.numpy(),
        "mse": mse.numpy(),
        "rmse": rmse.numpy(),
        "huber_loss": huber_loss.numpy(),
        "mase": mase.numpy()
    }

In [ ]:
forecast  = N_BEATS_MODEL.predict(feature)
preds =(forecast)
# labels.shape, feature

In [ ]:
evaluate_preds(y_true= labels, y_pred= preds)

In [ ]:
# model.save('model_11_multivariate_parameter_1_500.h5')

In [23]:
N_BEATS_MODEL.save('trained_model_11111.h5')

ValueError: Unable to synchronously create dataset (name already exists)

In [24]:
import os

# Get the current working directory
current_directory = os.getcwd()
print("Current working directory:", current_directory)

# List the contents of the working directory
contents = os.listdir(current_directory)
print("Contents of the working directory:", contents)

# Change directory to /kaggle/working/
os.chdir('/kaggle/working/')

# Get the current working directory (should be /kaggle/working/)
new_directory = os.getcwd()
print("New working directory:", new_directory)

# List the contents of the new working directory
new_contents = os.listdir(new_directory)
print("Contents of the new working directory:", new_contents)


Current working directory: /kaggle/working
Contents of the working directory: ['.virtual_documents', 'Model', 'trained_model.h5', 'trained_model_11111.h5']
New working directory: /kaggle/working
Contents of the new working directory: ['.virtual_documents', 'Model', 'trained_model.h5', 'trained_model_11111.h5']


In [1]:
pip list

Package                                  Version
---------------------------------------- -------------------
absl-py                                  1.4.0
accelerate                               0.29.3
access                                   1.1.9
affine                                   2.4.0
aiobotocore                              2.12.3
aiofiles                                 22.1.0
aiohttp                                  3.9.1
aiohttp-cors                             0.7.0
aioitertools                             0.11.0
aiorwlock                                1.3.0
aiosignal                                1.3.1
aiosqlite                                0.19.0
albumentations                           1.4.0
alembic                                  1.13.1
altair                                   5.3.0
annotated-types                          0.6.0
annoy                                    1.17.3
anyio                                    4.2.0
apache-beam                          